In [43]:
# DEVOIR 2
# PARTIE III: PRÉDICTIONS
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from itertools import combinations

In [20]:
features = np.arange(40)

In [56]:
# RÉGRESSION LINÉAIRE
# On mesure la qualité de la régression linéaire à l'aide du risque empirique
# càd la moyenne des risques (erreur quadratique). Le modèle de meilleure qualité
# lorsque son risque empirique est le plus petit.

# On standardise les données pour comparer les différents modèles.

dataB = pd.read_csv('dataB.csv', index_col=0)
#dataB = dataB.values
dataB = StandardScaler().fit_transform(dataB.values)

In [29]:
def risk_regression(x, y):
    # Séparer les données en données d'apprentissage et en données de test (ce qu'on veut prédire)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 1/5, random_state = 0)

    model = LinearRegression()
    model.fit(x_train, y_train)

    y_predict = model.predict(x_test)

    # Ici, l'erreur quadratique est notre fonction de perte
    risk = mean_squared_error(y_test, y_predict)
    return risk

In [36]:
# a) Qualité des prédicteurs en fonction des autres colonnes
# Exemple pour une seule colonne

risk_array_a = np.zeros(features.shape[0])

for i in range(features.shape[0]):
    x = np.delete(dataB, i, 1)
    y = dataB[:,i]
    risk_array_a[i] = risk_regression(x,y)

data_linear_a = pd.DataFrame(risk_array_a)
print(f'L\'attribut avec le plus petit risque empirique est {np.argmin(risk_array_a)}.')
# 24: Âge médian

L'attribut avec le plus petit risque empirique est 24.


In [47]:
# b) Paire de colonnes donnant les meilleures prédictions pour une colonne
matrix_b = np.zeros((features.shape[0],2), dtype = int)

for i in range(features.shape[0]):
    other_columns = np.delete(features, i)
    pairs = list(combinations(other_columns, 2))
    risk_array_b = np.zeros(len(pairs))
    
    # Pour chaque paire
    for j in range(len(pairs)):
        x = dataB[:,pairs[j]]
        y = dataB[:,i]
        risk_array_b[j] = risk_regression(x,y)
        
    min_index = np.argmin(risk_array_b)
    min_pairs = np.array(pairs[min_index])
    
    matrix_b[i][0] = min_pairs[0]
    matrix_b[i][1] = min_pairs[1]
    print(f'Caractéristique {i} terminée!')

Caractéristique 0 terminée!
Caractéristique 1 terminée!
Caractéristique 2 terminée!
Caractéristique 3 terminée!
Caractéristique 4 terminée!
Caractéristique 5 terminée!
Caractéristique 6 terminée!
Caractéristique 7 terminée!
Caractéristique 8 terminée!
Caractéristique 9 terminée!
Caractéristique 10 terminée!
Caractéristique 11 terminée!
Caractéristique 12 terminée!
Caractéristique 13 terminée!
Caractéristique 14 terminée!
Caractéristique 15 terminée!
Caractéristique 16 terminée!
Caractéristique 17 terminée!
Caractéristique 18 terminée!
Caractéristique 19 terminée!
Caractéristique 20 terminée!
Caractéristique 21 terminée!
Caractéristique 22 terminée!
Caractéristique 23 terminée!
Caractéristique 24 terminée!
Caractéristique 25 terminée!
Caractéristique 26 terminée!
Caractéristique 27 terminée!
Caractéristique 28 terminée!
Caractéristique 29 terminée!
Caractéristique 30 terminée!
Caractéristique 31 terminée!
Caractéristique 32 terminée!
Caractéristique 33 terminée!
Caractéristique 34 termi

In [48]:
# Mettre matrix_b dans le fichier .json pour la régression linéaire
linear_list = matrix_b.tolist()
with open('lineaire_paires_colonnes.json', 'w') as f:
    json.dump(linear_list, f)

In [60]:
# c) Paire de colonnes donnant le meilleur prédicteur pour chacune des caractéristiques en moyenne

opt_index = 0
opt_pairs = np.zeros(2, dtype=int)

pairs = list(combinations(features, 2))
risk_pairs = np.zeros(len(pairs))

for i in range(len(pairs)):
    if i % 100 == 0:
        print(f'Itération {i} complétée!')
    
    predicted_columns = np.delete(features, pairs[i])
    risk_array_c = np.zeros(predicted_columns.shape[0])
    
    for j in range(predicted_columns.shape[0]):
        x = dataB[:,pairs[i]]
        y = dataB[:,j]
        risk_array_c[j] = risk_regression(x,y)
    
    risk_pairs[i] = np.mean(risk_array_c)

opt_index = np.argmin(risk_pairs)
opt_pairs = np.array(pairs[opt_index])

Itération 0 complétée!
Itération 100 complétée!
Itération 200 complétée!
Itération 300 complétée!
Itération 400 complétée!
Itération 500 complétée!
Itération 600 complétée!
Itération 700 complétée!


In [64]:
opt_pairs
# 11: % de bouddhistes
# 24: Âge médian

array([11, 24])

In [75]:
# CLASSIFIEUR BAYÉSIEN
dataC = pd.read_csv('dataC.csv', index_col=0)
dataC = dataC.values

In [76]:
def risk_bayes(x, y):
    # Séparer les données en données d'apprentissage et en données de test (ce qu'on veut prédire)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 1/5, random_state = 0)

    # On utilise distribution multinomiale comme calcul de vraisemblance
    model = MultinomialNB()
    model.fit(x_train, y_train)

    y_predict = model.predict(x_test)
    risk = 1 - accuracy_score(y_test, y_predict)
    return risk

In [77]:
# a) Qualité des prédicteurs en fonction des autres colonnes
# Exemple pour une seule colonne

risk_array_a = np.zeros(features.shape[0])

for i in range(features.shape[0]):
    x = np.delete(dataC, i, 1)
    y = dataC[:,i]
    risk_array_a[i] = risk_bayes(x,y)

data_linear_a = pd.DataFrame(risk_array_a)
print(f'L\'attribut avec le plus petit risque empirique est {np.argmin(risk_array_a)}.')
# 5: Human Development Index

L'attribut avec le plus petit risque empirique est 5.


In [79]:
# b) Paire de colonnes donnant les meilleures prédictions pour une colonne
matrix_b = np.zeros((features.shape[0],2), dtype = int)

for i in range(features.shape[0]):
    other_columns = np.delete(features, i)
    pairs = list(combinations(other_columns, 2))
    risk_array_b = np.zeros(len(pairs))
    
    # Pour chaque paire
    for j in range(len(pairs)):
        x = dataC[:,pairs[j]]
        y = dataC[:,i]
        risk_array_b[j] = risk_bayes(x,y)
        
    min_index = np.argmin(risk_array_b)
    min_pairs = np.array(pairs[min_index])
    
    matrix_b[i][0] = min_pairs[0]
    matrix_b[i][1] = min_pairs[1]
    print(f'Caractéristique {i} terminée!')

Caractéristique 0 terminée!
Caractéristique 1 terminée!
Caractéristique 2 terminée!
Caractéristique 3 terminée!
Caractéristique 4 terminée!
Caractéristique 5 terminée!
Caractéristique 6 terminée!
Caractéristique 7 terminée!
Caractéristique 8 terminée!
Caractéristique 9 terminée!
Caractéristique 10 terminée!
Caractéristique 11 terminée!
Caractéristique 12 terminée!
Caractéristique 13 terminée!
Caractéristique 14 terminée!
Caractéristique 15 terminée!
Caractéristique 16 terminée!
Caractéristique 17 terminée!
Caractéristique 18 terminée!
Caractéristique 19 terminée!
Caractéristique 20 terminée!
Caractéristique 21 terminée!
Caractéristique 22 terminée!
Caractéristique 23 terminée!
Caractéristique 24 terminée!
Caractéristique 25 terminée!
Caractéristique 26 terminée!
Caractéristique 27 terminée!
Caractéristique 28 terminée!
Caractéristique 29 terminée!
Caractéristique 30 terminée!
Caractéristique 31 terminée!
Caractéristique 32 terminée!
Caractéristique 33 terminée!
Caractéristique 34 termi

In [80]:
# Mettre matrix_b dans le fichier .json pour le classifieur bayésien
linear_list = matrix_b.tolist()
with open('bayesien_paires_colonnes.json', 'w') as f:
    json.dump(linear_list, f)

In [81]:
# c) Paire de colonnes donnant le meilleur prédicteur pour chacune des caractéristiques en moyenne

opt_index = 0
opt_pairs = np.zeros(2, dtype=int)

pairs = list(combinations(features, 2))
risk_pairs = np.zeros(len(pairs))

for i in range(len(pairs)):
    if i % 100 == 0:
        print(f'Itération {i} complétée!')
    
    predicted_columns = np.delete(features, pairs[i])
    risk_array_c = np.zeros(predicted_columns.shape[0])
    
    for j in range(predicted_columns.shape[0]):
        x = dataC[:,pairs[i]]
        y = dataC[:,j]
        risk_array_c[j] = risk_bayes(x,y)
    
    risk_pairs[i] = np.mean(risk_array_c)

opt_index = np.argmin(risk_pairs)
opt_pairs = np.array(pairs[opt_index])

Itération 0 complétée!
Itération 100 complétée!
Itération 200 complétée!
Itération 300 complétée!
Itération 400 complétée!
Itération 500 complétée!
Itération 600 complétée!
Itération 700 complétée!


In [82]:
opt_pairs
# 5: Human Development Index
# 20: Fertility Rate

array([ 5, 20])